# Импорт библиотек и загрузка данных

In [15]:
import json
import cv2 
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import seaborn as sns
import pandas as pd
from collections import Counter
from functools import lru_cache
import os
import shutil
import re
import logging
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import datasets, transforms, models
from torch import nn
from torch import optim
from tqdm import tqdm
import torch.nn.functional as F
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
import torch

## Считываем данные

In [49]:
class ImageDataset:
    def __init__(self, path, path2image, n_classes=15):
        # Загрузка данных из CSV
        self.df = pd.read_csv(path)
        self.path2image = Path(path2image)
        self.N = n_classes
        
    def __len__(self):
        return len(self.df)

    def _encode_label(self, classes):
        label = np.zeros(self.N)
        for c in np.array(classes, dtype=np.int16) - 1:
            label[c] = 1
        return label
        
    def sample(self, clss=1, count=15):
        files = []
        while len(files) < count:
            indx = np.random.randint(0, len(self.df))
            labels = self.df.iloc[indx]["OUTPUT:classes"]
            try:
                if clss in self.parse(labels):
                    files.append(self.path2image / self.file_name(indx))
            except:
                continue
        return files
    
    def create_path(self, indx):
        return str(self.path2image / self.file_name(indx))
        
    def __getitem__(self, indx):
        img = cv2.imread(self.create_path(indx))
        clases = self.parse(self.df.iloc[indx]["OUTPUT:classes"])
        label = self._encode_label(clases)
        sample = {'image': img,
                  'label': label}
        return sample
            
    def file_name(self, indx):
        return self.df.iloc[indx]['file_name']
    
    def resize(self, img, scale=2):    
        shape = (np.array(img.shape[:2]) / scale).astype(np.int32)
        return cv2.resize(img, shape)
    
    def parse(self, st):        
        # Преобразуем строку вида '[2, 5, 11]' в список [2, 5, 11]
        return [int(x) for x in re.findall("\d+", st)]

<>:49: SyntaxWarning: invalid escape sequence '\d'
<>:49: SyntaxWarning: invalid escape sequence '\d'
C:\Users\maksi\AppData\Local\Temp\ipykernel_9316\1111695564.py:49: SyntaxWarning: invalid escape sequence '\d'
  return [int(x) for x in re.findall("\d+", st)]


In [50]:
class TorchImageDataset(ImageDataset, Dataset):
    def __init__(self, path, path2image, imgsz=256, MEAN = (0.485, 0.456, 0.406), STD = (0.229, 0.224, 0.225)):
        super().__init__(path, path2image)
        self.imgsz = imgsz
        self.fransform_img= transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Resize((self.imgsz, self.imgsz)),
                    transforms.RandomHorizontalFlip(),
                    transforms.RandomRotation(10),    
                    transforms.ColorJitter(brightness=0.2, 
                                                   contrast=0.2, 
                                                   saturation=0.2, 
                                                   hue=0.2),
                    transforms.Normalize(mean=MEAN, std=STD),
        ])
            
    @lru_cache(10000)
    def __getitem__(self,indx):
        img, label = super().__getitem__(indx).values()
        if img is not None:
            img = self.fransform_img(img)
            return {'image':img,'label':label}

## Подготовка файла

## Получаем датасет

In [51]:
dataset = TorchImageDataset("./analis/data.csv",  "../Resize/")

## DataLoader

In [52]:
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():  
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device       

    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            img,label = tuple(b.values())
            img = to_device(img,self.device)
            label = to_device(label, self.device)
            yield {"image":img,"label":label}

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [53]:
def split_data(dataset, train_ratio=0.7, val_ratio=0.1):

    total_size = len(dataset)
    train_size = int(train_ratio * total_size)
    val_size = int(val_ratio * total_size)
    test_size = total_size - train_size - val_size

    train_data, val_data, test_data = random_split(dataset, [train_size, val_size, test_size])

    return train_data, val_data, test_data

In [71]:
train_data, val_data, test_data = split_data(dataset, train_ratio=0.6, val_ratio=0.05)

In [55]:
type(dataset[0])

dict

In [56]:
device = torch.device("cuda:0")

In [57]:
len(dataset)

14594

In [72]:
train_loader = DeviceDataLoader(DataLoader(train_data, batch_size=16,num_workers = 0), device=device)
val_loader   = DeviceDataLoader(DataLoader(val_data, batch_size=16,num_workers = 0), device=device)
test_loader   = DeviceDataLoader(DataLoader(test_data, batch_size=16,num_workers = 0), device=device)

# Обучение модели

In [ ]:
class ModelMultilabel(nn.Module):
    def __init__(self, n_classes):
        super(ModelMultilabel, self).__init__()
        self.n_classes = n_classes
        
        self.model = timm.create_model('resnet18', pretrained=True)
        
        self.model.fc = nn.Identity()
        
        self.fc = nn.Linear(512, self.n_classes) 
        
    def forward(self, x):

        x = self.model(x)

        x = self.fc(x)
        
        return x

def train_model(model, dataloader_train, dataloader_valid, learningRate, num_epochs, device = 'cuda:0'):
    logger = logging.getLogger('api_log')
    logger.setLevel(logging.INFO)
    file_handler = logging.FileHandler(f'log_{learningRate}.txt')
    file_handler.setLevel(logging.DEBUG)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    file_handler.setFormatter(formatter)
    logger.addHandler(file_handler)
    logger.debug(f"Start_train  lr={learningRate}")
    
    # Установка оптимизатора с текущей скоростью обучения
    optimizer = optim.Adam(model.parameters(), lr=learningRate)
    criterion = nn.CrossEntropyLoss()
    # Цикл обучения
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        all_labels = []
        all_predictions = []

        for s in tqdm(dataloader_train, desc=f'Training Epoch {epoch+1}/{num_epochs}'):
            if s is None:
                continue
            inputs = s['image']
            labels = s['label']
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)  # Выходы модели
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

            predicted = outputs
            all_labels.extend(labels.detach().cpu().numpy())
            all_predictions.extend(predicted.detach().cpu().numpy())
        # Средняя потеря за эпоху
        epoch_loss = running_loss / len(dataloader_train)
        # precision = precision_score(all_labels, all_predictions, average='weighted')
        # recall = recall_score(all_labels, all_predictions, average='weighted')
        # accuracy = accuracy_score(all_labels, all_predictions)
        f_score = F_score(all_predictions, all_labels)
        # print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {epoch_loss:.4f}, "
        #       f"Recall: {recall:.4f}, Precision: {precision:.4f}, Accuracy: {accuracy:.4f}, F_score: {f_score:.4f}")
        logger.info(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {epoch_loss:.4f}, F_score: {f_score:.4f}")
        
        # logger.debug(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {epoch_loss:.4f}, Recall: {recall:.4f}, Precision: {precision:.4f}, Accuracy: {accuracy:.4f},  F_score: {f_score:.4f}")
        validate_model(model, dataloader_valid, logger)    
    # Сохранение модели
    torch.save(model.state_dict(), f'model_{learningRate}.pth')
    
def validate_model(model, dataloader_valid, logger, device='cuda:0'):
    model.eval()
    all_labels = []
    all_predictions = []
    with torch.no_grad():
        for s in tqdm(dataloader_valid, desc='Validation'):
            if s is None:
                continue
            inputs = s['image']
            labels = s['label']
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            predicted = outputs
            all_labels.extend(labels.detach().cpu().numpy())
            all_predictions.extend(predicted.detach().cpu().numpy())

    # precision = precision_score(all_labels, all_predictions, average='weighted')
    # recall = recall_score(all_labels, all_predictions, average='weighted')
    # accuracy = accuracy_score(all_labels, all_predictions)
    f_score = F_score(all_predictions, all_labels)
    # print(f"Validation Results - Recall: {recall:.4f}, Precision: {precision:.4f}, Accuracy: {accuracy:.4f}")
    # logger.debug(f"Validation Results - Recall: {recall:.4f}, Precision: {precision:.4f}, Accuracy: {accuracy:.4f}, F_score: {f_score:.4f}")
    logger.info(f"Validation Results -  F_score: {f_score:.4f}")


def F_score(output, label, threshold=0.5, beta=1): #Calculate the accuracy of the model
    if isinstance(output, list):
        output = np.array(output)
    if isinstance(label, list):
        label = np.array(label)
    print(label.shape, output.shape)
    assert (output.shape == label.shape),'shape is different'
    prob = output > threshold
    label = label > threshold

    TP = (prob == label).sum()
    TN = (np.bitwise_not(prob) == np.bitwise_not(label)).sum()
    FP = (prob == np.bitwise_not(label)).sum()
    FN = (np.bitwise_not(prob) == label).sum()
    print(TP, TN, FP, FN)
    precision = np.mean(TP / (TP + FP + 1e-12))
    recall = np.mean(TP / (TP + FN + 1e-12))
    F2 = (1 + beta**2) * precision * recall / (beta**2 * precision + recall + 1e-12)
    return F2


In [68]:
import timm

In [74]:
model = ModelMultilabel(15).to(device)

In [70]:
dataset[1]['label']

array([0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.])

In [75]:
train_model(model, train_loader, val_loader, 0.001, 50)

Training Epoch 1/50: 100%|██████████| 548/548 [10:57<00:00,  1.20s/it]


(8756, 15) (8756, 15)
111997 111997 19343 19343


Validation: 100%|██████████| 46/46 [01:37<00:00,  2.11s/it]


(729, 15) (729, 15)
9545 9545 1390 1390


Training Epoch 2/50: 100%|██████████| 548/548 [00:39<00:00, 13.90it/s]


...

(729, 15) (729, 15)
10037 10037 898 898


Training Epoch 50/50: 100%|██████████| 548/548 [00:39<00:00, 13.97it/s]


(8756, 15) (8756, 15)
131333 131333 7 7


Validation: 100%|██████████| 46/46 [00:01<00:00, 43.06it/s]


(729, 15) (729, 15)
10038 10038 897 897


In [78]:
torch.save(model, "model80.pth")

In [ ]:
model = torch.load("model.pth")
model.eval()

## Метрики

In [76]:
model.eval()
all_predictions = []
all_labels = []
with torch.no_grad():
    for batch in tqdm(test_loader, desc="Evaluating"):
        inputs = batch['image'].to(device)
        labels = batch['label'].to(device)

        outputs = model(inputs)
        outputs = (outputs > 0.5).float()

        all_predictions.append(outputs.cpu().numpy())
        all_labels.append(labels.cpu().numpy())
    
all_predictions = np.vstack(all_predictions)
all_labels = np.vstack(all_labels)
    
f1 = f1_score(all_labels, all_predictions, average='weighted')  
print(f"F1-Score: {f1:.4f}")

Evaluating: 100%|██████████| 320/320 [13:07<00:00,  2.46s/it]

F1-Score: 0.8061


In [79]:
f1_score(all_labels, all_predictions, average='macro')  

np.float64(0.6028597595650425)

In [80]:
f1_score(all_labels, all_predictions, average='micro')  

np.float64(0.818353419485988)

In [77]:
import numpy as np
from sklearn.metrics import f1_score
import torch
from tqdm import tqdm

# Переводим модель в режим оценки
model.eval()

all_predictions = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Evaluating"):
        inputs = batch['image'].to(device)
        labels = batch['label'].to(device)

        outputs = model(inputs)
        outputs = (outputs > 0.5).float()  # Если задача бинарной классификации, иначе можно использовать softmax

        all_predictions.append(outputs.cpu().numpy())
        all_labels.append(labels.cpu().numpy())

# Преобразуем списки в массивы
all_predictions = np.vstack(all_predictions)
all_labels = np.vstack(all_labels)

# Вычисляем F1-Score для каждого класса
f1_per_class = f1_score(all_labels, all_predictions, average=None)

# Выводим F1-Score для каждого класса
for i, f1 in enumerate(f1_per_class):
    print(f"Class {i} F1-Score: {f1:.4f}")

Evaluating: 100%|██████████| 320/320 [00:07<00:00, 42.48it/s]

Class 0 F1-Score: 0.7120
Class 1 F1-Score: 0.8665
Class 2 F1-Score: 0.4981
Class 3 F1-Score: 0.5759
Class 4 F1-Score: 0.9214
Class 5 F1-Score: 0.0551
Class 6 F1-Score: 0.8484
Class 7 F1-Score: 0.8639
Class 8 F1-Score: 0.4510
Class 9 F1-Score: 0.8189
Class 10 F1-Score: 0.8396
Class 11 F1-Score: 0.1928
Class 12 F1-Score: 0.6957
Class 13 F1-Score: 0.2500
Class 14 F1-Score: 0.4536
